In [76]:
import bs4 as bs
import requests
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_squared_error

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'

# Stock Price Data

In [3]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]

In [4]:
def process_single_stock(stock_data):
  full = pd.DataFrame()
  full['ma5'] = stock_data.rolling(5).mean()
  full['ma10'] = stock_data.rolling(10).mean()
  full['ma20'] = stock_data.rolling(20).mean()
  full['ma30'] = stock_data.rolling(30).mean()
  full['close'] = stock_data
  full = full.dropna(axis = 0)
  price_max = np.max(full['close'])
  return full / price_max

In [5]:
def process_all_stocks(tickers, start, end, steps):
  data = yf.download(tickers, start = start, end = end)
  data = data['Close']
  data = data.dropna(axis = 1)
  for i in range(len(data.columns)):
    single_EOD = process_single_stock(data.iloc[:, i])
    if (i == 0):
      eod_data = np.zeros([len(data.columns), single_EOD.shape[0], single_EOD.shape[1]])
      ground_truth = np.zeros([len(data.columns), single_EOD.shape[0]])
      close_prices = np.zeros([len(data.columns), single_EOD.shape[0]])
    eod_data[i, :, :] = single_EOD
    close_prices[i, :] = single_EOD.iloc[:, -1]
    for row in range(single_EOD.shape[0]):
      if (row > steps - 1):
        ground_truth[i, row] = (single_EOD.iloc[row][-1] - single_EOD.iloc[row - steps][-1]) / single_EOD.iloc[row - steps][-1]
  return data.columns, eod_data, close_prices, ground_truth

In [6]:
start = datetime.datetime(2014, 1, 1)
end = datetime.datetime(2024, 1, 1)
steps = 1

In [ ]:
stocks, eod_data, price_data, gt_data = process_all_stocks(tickers, start, end, steps)

In [ ]:
print("Number of Stocks: {}".format(len(stocks)))
print("Shape of EOD data: {}".format(eod_data.shape))
print("Shape of Price data: {}".format(price_data.shape))
print("Shape of Ground Truth data: {}".format(gt_data.shape))

# Stock Relation Data

In [9]:
def preprocess(df):
  returns = df.pct_change()
  returns = returns.drop(returns.index[0])
  returns = returns.dropna(axis = 1)
  return returns

In [82]:
def compute_simple_correlation(df, method):
  returns = preprocess(df)
  return returns.corr(method = method)

In [83]:
def compute_granger(df, maxlag):
  returns = preprocess(df)
  variables = returns.columns
  granger = pd.DataFrame(np.zeros((len(variables), len(variables))), columns = variables, index = variables)
  for row in granger.index:
    for col in granger.columns:
      test_result = grangercausalitytests(returns[[row, col]], maxlag, verbose = None)
      p_vals = [round(test_result[i + 1][0]["ssr_chi2test"][1], 3) for i in range(maxlag)]
      min_p_val = np.min(p_vals)
      granger.loc[row, col] = min_p_val
  return granger

In [ ]:
relation_start = datetime.datetime(2023, 1, 1)
relation_data = yf.download(stocks.tolist(), start = relation_start, end = end)
relation_data = relation_data["Close"]

In [70]:
pearson = compute_simple_correlation(relation_data, "pearson")
pearson[pearson == 1] = 0
pearson[abs(pearson) >= 0.5] = 1
pearson[abs(pearson) < 0.5] = 0
pearson = np.expand_dims(np.array(pearson), axis = 2)

In [73]:
spearman = compute_simple_correlation(relation_data, "spearman")
spearman[spearman == 1] = 0
spearman[abs(spearman) >= 0.5] = 1
spearman[abs(spearman) < 0.5] = 0
spearman = np.expand_dims(np.array(spearman), axis = 2)

In [ ]:
granger = compute_granger(relation_data, 5)
granger[granger == 1] = 0
granger[granger <= 0.05] = 1
granger[granger > 0.05] = 0
granger = np.expand_dims(np.array(granger), axis = 2)

# Rank LSTM

In [13]:
class RankLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(RankLSTM, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, batch_first = True)
    self.fc = nn.Linear(hidden_size, output_size)
    self.leaky_relu = nn.LeakyReLU(negative_slope = 0.2)

  def forward(self, input): # input: (batch size, sequence length, input size)
    x, _ = self.lstm(input) # x: (batch size, sequence length, hidden size)
    seq_embed = x[:, -1, :] # seq_embed: (batch size, hidden size)
    out = self.fc(seq_embed) # out: (batch size, output size)
    preds = self.leaky_relu(out) # preds: (batch size, output size)
    return preds

# Relational Stock Ranking

In [14]:
class GraphModule(nn.Module):
  def __init__(self, rel_encoding, input_size, directed):
    super(GraphModule, self).__init__()
    self.relation = nn.Parameter(torch.tensor(rel_encoding, dtype = torch.float32), requires_grad = False) # self.relation: (batch size, batch size, 1)
    self.all_one = nn.Parameter(torch.ones(len(stocks), 1, dtype = torch.float32), requires_grad = False) # self.all_one: (batch size, 1)
    self.rel_weight = nn.Linear(rel_encoding.shape[-1], 1)
    self.head_weight = nn.Linear(input_size, 1)
    self.directed = directed
    if (directed):
      self.tail_weight = nn.Linear(input_size, 1)
    self.leaky_relu = nn.LeakyReLU(negative_slope = 0.2)
    self.softmax = nn.Softmax(dim = 0)

  def forward(self, input): # input: (batch size, hidden size)
    rel_weight = self.leaky_relu(self.rel_weight(self.relation)) # rel_weight: (batch size, batch size, 1)
    rel_weight = rel_weight[:, :, -1] # rel_weight: (batch size, batch size)
    head_weight = self.leaky_relu(self.head_weight(input)) # head_weight: (batch size, 1)
    all_one = self.all_one
    if (self.directed):
      tail_weight = self.leaky_relu(self.tail_weight(input)) # tail_weight: (batch size, 1)
      weight = (head_weight @ all_one.t() + all_one @ tail_weight.t()) + rel_weight # weight: (batch size, batch size)
    else:
      weight = (head_weight @ all_one.t() + all_one @ head_weight.t()) + rel_weight # weight: (batch size, batch size)
    weight = self.softmax(weight) # weight: (batch size, batch size)
    output = weight @ input # output: (batch size, hidden size)
    return output

In [15]:
class RSR(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, rel_encoding, directed):
    super(RSR, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, batch_first = True)
    self.graph_layer = GraphModule(rel_encoding, hidden_size, directed)
    self.fc = nn.Linear(hidden_size * 2, output_size)
    self.leaky_relu = nn.LeakyReLU(negative_slope = 0.2)

  def forward(self, input): # input: (batch size, sequence length, input size)
    x, _ = self.lstm(input) # x: (batch size, sequence length, hidden size)
    seq_embed = x[:, -1, :] # seq_embed: (batch size, hidden size)
    rel_embed = self.graph_layer(seq_embed) # rel_embed: (batch size, hidden size)
    full_embed = torch.cat((seq_embed, rel_embed), dim = 1) # full_embed: (batch size, hidden_size * 2)
    out = self.fc(full_embed) # out: (batch size, output size)
    preds = self.leaky_relu(out) # preds: (batch size, output size)
    return preds

# Train, Tune, and Test

In [24]:
def get_batch(eod_data, price_data, gt_data, offset, seq_len, steps):
  eod_batch = eod_data[:, offset:offset + seq_len, :]
  price_batch = np.expand_dims(price_data[:, offset + seq_len - 1], axis = 1)
  gt_batch = np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis = 1)
  return eod_batch, price_batch, gt_batch

In [25]:
def get_loss(prediction, price_batch, gt_batch, alpha):
  return_ratio = torch.div(torch.sub(prediction, price_batch), price_batch)
  reg_loss = F.mse_loss(gt_batch, return_ratio)
  all_one = torch.ones(len(stocks), 1, dtype = torch.float32).to(device)
  pred_pw_diff = torch.sub(return_ratio @ all_one.t(), all_one @ return_ratio.t())
  gt_pw_diff = torch.sub(all_one @ gt_batch.t(), gt_batch @ all_one.t())
  rank_loss = torch.mean(F.relu(pred_pw_diff * gt_pw_diff))
  loss = reg_loss + alpha * rank_loss
  return loss, reg_loss, rank_loss, return_ratio

In [26]:
def evaluate(prediction, ground_truth):
  performance = {}
  performance["mse"] = mean_squared_error(prediction, ground_truth)
  mrr = 0
  bt_top1 = 0
  bt_top5 = 0
  bt_top10 = 0

  for i in range(prediction.shape[1]):
    rank_gt = np.argsort(ground_truth[:, i])
    rank_pred = np.argsort(prediction[:, i])
    pred_top1 = rank_pred[-1]
    pred_top5 = rank_pred[range(-1, -6, -1)]
    pred_top10 = rank_pred[range(-1, -11, -1)]

    # MRR for top 1
    top1_pos_in_gt = 0
    for j in range(1, prediction.shape[0] + 1):
      rank = rank_gt[-1 * j]
      top1_pos_in_gt += 1
      if (rank == pred_top1):
        break;
    mrr += 1.0 / top1_pos_in_gt

    # Back-testing for Investment Return Ratio
    real_rr_top1 = ground_truth[pred_top1][i]
    bt_top1 += real_rr_top1

    real_rr_top5 = 0
    for pred in pred_top5:
      real_rr_top5 += ground_truth[pred][i]
    real_rr_top5 /= 5
    bt_top5 += real_rr_top5

    real_rr_top10 = 0
    for pred in pred_top10:
      real_rr_top10 += ground_truth[pred][i]
    real_rr_top10 /= 10
    bt_top10 += real_rr_top10

  performance["mrr"] = mrr
  performance["bt_top1"] = bt_top1
  performance["bt_top5"] = bt_top5
  performance["bt_top10"] = bt_top10

  return performance

In [27]:
def validate(start_index, end_index, eod_data, price_data, gt_data):
  with torch.no_grad():
    pred = np.zeros((len(stocks), end_index - start_index), dtype = float)
    gt = np.zeros((len(stocks), end_index - start_index), dtype = float)
    test_loss = 0
    test_reg_loss = 0
    test_rank_loss = 0
    for offset in range(start_index - seq_len - steps + 1, end_index - seq_len - steps + 1):
      eod_batch, price_batch, gt_batch = map(lambda x: torch.Tensor(x).to(device),
                                             get_batch(eod_data, price_data, gt_data, offset, seq_len, steps))
      prediction = model(eod_batch)
      loss, reg_loss, rank_loss, return_ratio = get_loss(prediction, price_batch, gt_batch, alpha)
      test_loss += loss.item()
      test_reg_loss += reg_loss.item()
      test_rank_loss += rank_loss.item()
      pred[:, offset - (start_index - seq_len - steps + 1)] = return_ratio[:, 0].cpu()
      gt[:, offset - (start_index - seq_len - steps + 1)] = gt_batch[:, 0].cpu()
    test_loss = test_loss / (end_index - start_index)
    test_reg_loss = test_reg_loss / (end_index - start_index)
    test_rank_loss = test_rank_loss / (end_index - start_index)
    performance = evaluate(pred, gt)
  return test_loss, test_reg_loss, test_rank_loss, performance

In [28]:
seed = 229
val_split = 2000
test_split = 2240
offsets = np.arange(0, val_split)
input_size = 5
output_size = 1

In [56]:
# Rank LSTM

num_epochs = 50
seq_len = 16
alpha = 1
hidden_size = 16
model = RankLSTM(input_size, hidden_size, output_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-6)

In [74]:
# RSR

num_epochs = 50
seq_len = 16
alpha = 1
hidden_size = 16
model = RSR(input_size, hidden_size, output_size, spearman, False).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-6)

In [ ]:
np.random.seed(seed)
torch.random.manual_seed(seed)
for epoch in range(num_epochs):
  np.random.shuffle(offsets)
  train_loss = 0
  train_reg_loss = 0
  train_rank_loss = 0
  for i in range(val_split - seq_len - steps + 1):
    eod_batch, price_batch, gt_batch = map(lambda x: torch.Tensor(x).to(device),
                                           get_batch(eod_data, price_data, gt_data, offsets[i], seq_len, steps))
    optimizer.zero_grad()
    prediction = model(eod_batch)
    loss, reg_loss, rank_loss, _ = get_loss(prediction, price_batch, gt_batch, alpha)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
    train_reg_loss += reg_loss.item()
    train_rank_loss += rank_loss.item()

  print("Epoch: {}".format(epoch))

  # Train
  train_loss = train_loss / (val_split - seq_len - steps + 1)
  train_reg_loss = train_reg_loss / (val_split - seq_len - steps + 1)
  train_rank_loss = train_rank_loss/ (val_split - seq_len - steps + 1)
  print("\tTrain Loss: {}".format(train_loss))
  print("\tTrain Reg Loss: {}".format(train_reg_loss))
  print("\tTrain Rank Loss: {}".format(train_rank_loss))

  # Val
  val_loss, val_reg_loss, val_rank_loss, val_perform = validate(val_split, test_split, eod_data, price_data, gt_data)
  print("\tVal Loss: {}".format(val_loss))
  print("\tVal Reg Loss: {}".format(val_reg_loss))
  print("\tVal Rank Loss: {}".format(val_rank_loss))
  print("\tVal MSE: {}".format(val_perform["mse"]))
  print("\tVal MRR: {}".format(val_perform["mrr"]))
  print("\tVal Top 1: {}".format(val_perform["bt_top1"]))
  print("\tVal Top 5: {}".format(val_perform["bt_top5"]))
  print("\tVal Top 10: {}".format(val_perform["bt_top10"]))

  # Test
  test_loss, test_reg_loss, test_rank_loss, test_perform = validate(test_split, gt_data.shape[1], eod_data, price_data, gt_data)
  print("\tTest Loss: {}".format(test_loss))
  print("\tTest Reg Loss: {}".format(test_reg_loss))
  print("\tTest Rank Loss: {}".format(test_rank_loss))
  print("\tTest MSE: {}".format(test_perform["mse"]))
  print("\tTest MRR: {}".format(test_perform["mrr"]))
  print("\tTest Top 1: {}".format(test_perform["bt_top1"]))
  print("\tTest Top 5: {}".format(test_perform["bt_top5"]))
  print("\tTest Top 10: {}".format(test_perform["bt_top10"]))